<a href="https://colab.research.google.com/github/vidhi-sys/GenAI_LangChain_and_RAG/blob/main/Text_summarization_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Nov 17 15:05:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# Test basic imports
from transformers import pipeline
from datasets import load_dataset
!pip install sacrebleu
!pip install rouge-score
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

print("All packages imported successfully!")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a58a5006ed415d7fc0f2aa38d55c7f83f7a41406808ec70f81ac110e07cc3ba
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
All packages imported successfully!


In [6]:
! pip install transformers accelerate

In [9]:
from transformers import pipeline , set_seed
from datasets import load_dataset,load_from_disk
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import torch
from tqdm import tqdm



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
model_ckpt= "google/pegasus-cnn_dailymail"
tokenizer= AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

**Transformer based model**-pegasus for text summarization
[Hugging Face READ ](https://huggingface.co/google/pegasus-cnn_dailymail)
Dataset From samsum Corpus containing conversations and summary

In [12]:
my_model= AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [19]:
dataset=load_dataset("knkarthick/samsum")

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [22]:
dataset["train"]["dialogue"][1]

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [23]:
def convert_examples_to_feature(example):
  input_encodings= tokenizer(example["dialogue"],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings= tokenizer(example["summary"],max_length=128,truncation=True)
    return {"input_ids":input_encodings["input_ids"],
          "attention_mask":input_encodings["attention_mask"],
          "labels":target_encodings["input_ids"]}

In [24]:
dataset_samsum_pt=dataset.map(convert_examples_to_feature,batched=True)

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [25]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

**TRAINING **

In [26]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator= DataCollatorForSeq2Seq(tokenizer,model=my_model)

In [30]:
!pip install --upgrade transformers
from transformers import TrainingArguments,Trainer
trainer_args= TrainingArguments(
    output_dir="pegasus-samsum",num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    # Removed evaluation_strategy and eval_steps to bypass TypeError related to old transformers version in runtime
    # evaluation_strategy="steps",eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [31]:
trainer= Trainer(model=my_model,args=trainer_args,
                  tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

/tmp/ipython-input-1385707652.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer= Trainer(model=my_model,args=trainer_args,


In [33]:
trainer.train()

KeyboardInterrupt: 

**EVALUATION**

In [35]:
def generate_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]
def calculate_metric_on_test_ds(dataset,metric,model,tokenizer , batch_size=16,device='cuda',
                                column_summary="highlights"):
  article_batches=list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches=list(generate_batch_sized_chunks(dataset[column_summary],batch_size))
  for article_batch,target_batch in tqdm(
      zip(article_batches,target_batches),total=len(article_batches)
  ):
    inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")
    summaries=model.generate(input_ids=inputs["input_ids"].to(device),
                             attention_mask=inputs["attention_mask"].to(device),
                             length_penalty=0.8,num_beams=8,max_length=128)
    decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True)
                       for s in summaries]
    decoded_summaries=[d.replace(""," ") for d in decoded_summaries]
    metric.add_batch(predictions=decoded_summaries,refrences=target_batch)
    score =metric.compute()
    return score

In [37]:
gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":128}
sample_text=dataset["test"][0]["dialogue"]
reference=dataset["test"][0]["summary"]
pipe=pipeline("summarization",model=my_model,tokenizer=tokenizer)
pipe_out=pipe(sample_text,**gen_kwargs)[0]["summary_text"]
print(f"Dialogue: \n{sample_text}")
print(f"Summary: \n{reference}")

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue: 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Summary: 
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
